In [ ]:
!pip install rdkit padelpy lightgbm scikit-learn scikit-optimize

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from padelpy import padeldescriptor
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 9.1 MB/s eta 0:00:00


In [ ]:
import os

!fusermount -u /content/drive

!rm -rf /content/drive/*


fusermount: failed to unmount /content/drive: No such file or directory


In [ ]:

import os, time
from google.colab import drive
drive.mount('/content/drive')
WORKDIR = "/content/drive/MyDrive/moltox_project"
os.makedirs(WORKDIR, exist_ok=True)
print("Working dir:", WORKDIR)


Mounted at /content/drive
Working dir: /content/drive/MyDrive/moltox_project


In [ ]:

!pip install --quiet pubchempy requests tqdm


import importlib, sys
for pkg in ("requests","pubchempy","tqdm"):
    try:
        importlib.import_module(pkg)
        print(pkg, "OK")
    except Exception as e:
        print(pkg, "FAILED:", e)
        raise

import requests
r = requests.get("http://rest.kegg.jp/list/drug", timeout=30)
print("KEGG request status:", r.status_code)
print("Sample lines:", r.text.splitlines()[:3])


requests OK
pubchempy OK
tqdm OK
KEGG request status: 200
Sample lines: ['D00001\tWater (JP18/USP); Purified water (JP18); Purified water in containers (JP18); Water, purified (USP); Sterile purified water in containers (JP18); Water for injection (JP18); Water for injection in containers (JP18); Sterile water (TN)', 'D00002\tNadide (JAN/USAN/INN); Nicotinamide adenine dinucleotide', 'D00003\tOxygen (JP18/USP)']


In [ ]:
import requests, pubchempy as pcp
from time import sleep
from tqdm import tqdm


r = requests.get("http://rest.kegg.jp/list/drug", timeout=30)
lines = r.text.splitlines()
print("KEGG drugs lines:", len(lines))


kegg_entries = []
for L in lines:
    try:
        kid, name = L.split("\t",1)

        name = name.split(";")[0].split(",")[0].strip()
        kegg_entries.append((kid, name))
    except:
        pass

print("Parsed KEGG drugs:", len(kegg_entries))


KEGG_LIMIT = 2000
mapped = []
for i,(kid, name) in enumerate(tqdm(kegg_entries[:KEGG_LIMIT], desc="mapping KEGG->PubChem")):
    try:
        comps = pcp.get_compounds(name, 'name')
        if comps:
            cid = comps[0].cid
            smi = comps[0].canonical_smiles
            mapped.append({'source':'KEGG','kegg_id':kid,'name':name,'CID':cid,'SMILES':smi})
    except Exception as e:
        pass
    sleep(0.12)

df_kegg = pd.DataFrame(mapped)
print("Mapped KEGG entries:", len(df_kegg))
df_kegg.to_csv(os.path.join(WORKDIR,"kegg_mapped_pubchem.csv"), index=False)
df_kegg.head()


KEGG drugs lines: 12684
Parsed KEGG drugs: 12684


mapping KEGG->PubChem:   0%|          | 0/2000 [00:00<?, ?it/s]/tmp/ipython-input-3954121770.py:31: PubChemPyDeprecationWarning: canonical_smiles is deprecated: Use connectivity_smiles instead
  smi = comps[0].canonical_smiles
mapping KEGG->PubChem: 100%|██████████| 2000/2000 [28:51<00:00,  1.15it/s]


Mapped KEGG entries: 1636


,source,kegg_id,name,CID,SMILES
0,KEGG,D00001,Water (JP18/USP),962,O
1,KEGG,D00002,Nadide (JAN/USAN/INN),5893,C1=CC(=C[N+](=C1)C2C(C(C(O2)COP(=O)(O)OP(=O)(O...
2,KEGG,D00003,Oxygen (JP18/USP),977,O=O
3,KEGG,D00004,Carbon dioxide (JP18/USP),280,C(=O)=O
4,KEGG,D00005,Flavin adenine dinucleotide (JAN),643975,CC1=CC2=C(C=C1C)N(C3=NC(=O)NC(=O)C3=N2)CC(C(C(...


In [ ]:
from google.colab import files
import os, pandas as pd
from rdkit import Chem

WORKDIR = "/content/drive/MyDrive/moltox_project"
os.makedirs(WORKDIR, exist_ok=True)

uploaded = files.upload()
fname = list(uploaded.keys())[0]


suppl = Chem.SDMolSupplier(fname)
smiles = []
for m in suppl:
    if m is None:
        continue
    try:
        s = Chem.MolToSmiles(m, isomericSmiles=True)
        atoms = [a.GetSymbol() for a in m.GetAtoms()]
        if 'C' in atoms:
            smiles.append(s)
    except:
        pass


smiles = list(dict.fromkeys(smiles))
print("Extracted toxic SMILES from SDF:", len(smiles))

df_tox = pd.DataFrame({'SMILES_can': smiles, 'label': 1})
out = os.path.join(WORKDIR, "t3db_canon_from_sdf.csv")
df_tox.to_csv(out, index=False)
print("Saved to", out)


Saving structures.sdf to structures.sdf


[21:53:19] Explicit valence for atom # 1 N, 3, is greater than permitted
[21:53:19] ERROR: Could not sanitize molecule ending on line 3111
[21:53:19] ERROR: Explicit valence for atom # 1 N, 3, is greater than permitted
[21:53:20] WARNING: not removing hydrogen atom without neighbors
[21:53:20] WARNING: not removing hydrogen atom without neighbors
[21:53:20] Explicit valence for atom # 0 Ar, 1, is greater than permitted
[21:53:20] ERROR: Could not sanitize molecule ending on line 83908
[21:53:20] ERROR: Explicit valence for atom # 0 Ar, 1, is greater than permitted
[21:53:20] WARNING: not removing hydrogen atom without neighbors
[21:53:20] WARNING: not removing hydrogen atom without neighbors
[21:53:20] WARNING: not removing hydrogen atom without neighbors
[21:53:20] Explicit valence for atom # 0 O, 3, is greater than permitted
[21:53:20] ERROR: Could not sanitize molecule ending on line 153360
[21:53:20] ERROR: Explicit valence for atom # 0 O, 3, is greater than permitted
[21:53:20] Ex

Extracted toxic SMILES from SDF: 2902
Saved to /content/drive/MyDrive/moltox_project/t3db_canon_from_sdf.csv


In [ ]:

import os, pandas as pd
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

WORKDIR = os.getenv("WORKDIR", "/content/drive/MyDrive/moltox_project")
os.makedirs(WORKDIR, exist_ok=True)

kegg_path = os.path.join(WORKDIR, "kegg_mapped_pubchem.csv")
pub_tox_path = os.path.join(WORKDIR, "tox_pubchem_smiles.csv")
t3_csv = os.path.join(WORKDIR, "t3db_canon_from_csv.csv")
t3_sdf = os.path.join(WORKDIR, "t3db_canon_from_sdf.csv")


if os.path.exists(kegg_path):
    df_non = pd.read_csv(kegg_path, low_memory=False)

    if 'SMILES' in df_non.columns and 'SMILES_can' not in df_non.columns:
        df_non['SMILES_can'] = df_non['SMILES']
else:
    print("WARNING: KEGG file not found at", kegg_path)
    df_non = pd.DataFrame(columns=['SMILES_can'])


if os.path.exists(pub_tox_path):
    df_tox_pub = pd.read_csv(pub_tox_path, low_memory=False)
    if 'SMILES' in df_tox_pub.columns and 'SMILES_can' not in df_tox_pub.columns:
        df_tox_pub['SMILES_can'] = df_tox_pub['SMILES']
else:
    df_tox_pub = pd.DataFrame(columns=['SMILES_can'])

if os.path.exists(t3_csv):
    df_t3 = pd.read_csv(t3_csv, low_memory=False)
    if 'SMILES_can' not in df_t3.columns and 'SMILES' in df_t3.columns:
        df_t3['SMILES_can'] = df_t3['SMILES']
elif os.path.exists(t3_sdf):
    df_t3 = pd.read_csv(t3_sdf, low_memory=False)
    if 'SMILES_can' not in df_t3.columns and 'SMILES' in df_t3.columns:
        df_t3['SMILES_can'] = df_t3['SMILES']
else:
    df_t3 = pd.DataFrame(columns=['SMILES_can'])

def canonicalize_and_filter_series(series):
    out = []
    for s in series.dropna().astype(str):
        m = Chem.MolFromSmiles(s)
        if m is None:
            continue
        atoms = [a.GetSymbol() for a in m.GetAtoms()]
        if 'C' not in atoms and 'c' not in atoms:
            continue
        out.append(Chem.MolToSmiles(m, isomericSmiles=True))
    # unique preserve order
    seen = set()
    unique = []
    for x in out:
        if x not in seen:
            seen.add(x)
            unique.append(x)
    return unique


print("Canonicalizing sources...")
non_smiles = []
if 'SMILES_can' in df_non.columns:
    non_smiles = canonicalize_and_filter_series(df_non['SMILES_can'])
    df_non_clean = pd.DataFrame({'SMILES_can': non_smiles, 'label': 0})
else:
    df_non_clean = pd.DataFrame(columns=['SMILES_can','label'])

tox_pub_smiles = []
if 'SMILES_can' in df_tox_pub.columns:
    tox_pub_smiles = canonicalize_and_filter_series(df_tox_pub['SMILES_can'])
    df_tox_pub_clean = pd.DataFrame({'SMILES_can': tox_pub_smiles, 'label': 1})
else:
    df_tox_pub_clean = pd.DataFrame(columns=['SMILES_can','label'])

t3_smiles = []
if 'SMILES_can' in df_t3.columns:
    t3_smiles = canonicalize_and_filter_series(df_t3['SMILES_can'])
    df_t3_clean = pd.DataFrame({'SMILES_can': t3_smiles, 'label': 1})
else:
    df_t3_clean = pd.DataFrame(columns=['SMILES_can','label'])


df_toxic = pd.concat([df_tox_pub_clean, df_t3_clean], ignore_index=True).drop_duplicates(subset=['SMILES_can']).reset_index(drop=True)
df_non_toxic = df_non_clean.drop_duplicates(subset=['SMILES_can']).reset_index(drop=True)

print("Counts -> KEGG non-toxic:", len(df_non_toxic), "PubChem toxic:", len(df_tox_pub_clean), "T3DB:", len(df_t3_clean))
combined = pd.concat([df_toxic[['SMILES_can','label']], df_non_toxic[['SMILES_can','label']]], ignore_index=True)
combined = combined.drop_duplicates(subset=['SMILES_can']).reset_index(drop=True)
print("Combined dataset size:", len(combined))

outpath = os.path.join(WORKDIR, "moltox_raw_combined.csv")
combined.to_csv(outpath, index=False)
print("Saved combined file to:", outpath)
combined.head(10)


Canonicalizing sources...
Counts -> KEGG non-toxic: 1595 PubChem toxic: 0 T3DB: 0
Combined dataset size: 1595
Saved combined file to: /content/drive/MyDrive/moltox_project/moltox_raw_combined.csv


,SMILES_can,label
0,NC(=O)c1ccc[n+](C2OC(COP(=O)(O)OP(=O)(O)OCC3OC...,0
1,O=C=O,0
2,Cc1cc2nc3c(=O)[nH]c(=O)nc-3n(CC(O)C(O)C(O)COP(...,0
3,NC(CCC(=O)O)C(=O)O,0
4,CC(=O)O,0
5,NCC(=O)O,0
6,CC(N)C(=O)O,0
7,NC(CC(=O)O)C(=O)O,0
8,NC(CCC(=O)NC(CS)C(=O)NCC(=O)O)C(=O)O,0
9,NC(=O)CCC(N)C(=O)O,0


In [ ]:
import os, pandas as pd

WORKDIR = "/content/drive/MyDrive/moltox_project"


df_kegg = pd.read_csv(os.path.join(WORKDIR, "kegg_mapped_pubchem.csv"))
if 'SMILES' in df_kegg.columns and 'SMILES_can' not in df_kegg.columns:
    df_kegg['SMILES_can'] = df_kegg['SMILES']
df_kegg = df_kegg[['SMILES_can']].dropna().drop_duplicates()
df_kegg['label'] = 0
print("KEGG molecules:", df_kegg.shape)

df_tox = pd.read_csv(os.path.join(WORKDIR, "t3db_canon_from_sdf.csv"))
print("T3DB molecules:", df_tox.shape)


df_all = pd.concat([df_kegg, df_tox], ignore_index=True).drop_duplicates(subset=['SMILES_can'])
print("Combined molecules:", df_all.shape)
print("Label distribution:", df_all['label'].value_counts())

outpath = os.path.join(WORKDIR, "moltox_raw_combined.csv")
df_all.to_csv(outpath, index=False)
print("Saved combined dataset to", outpath)


KEGG molecules: (1624, 2)
T3DB molecules: (2902, 2)
Combined molecules: (4518, 2)
Label distribution: label
1    2894
0    1624
Name: count, dtype: int64
Saved combined dataset to /content/drive/MyDrive/moltox_project/moltox_raw_combined.csv


In [ ]:

import os, pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from tqdm import tqdm

WORKDIR = os.getenv("WORKDIR", "/content/drive/MyDrive/moltox_project")
inp = os.path.join(WORKDIR, "moltox_raw_combined.csv")
if not os.path.exists(inp):
    raise FileNotFoundError(f"{inp} not found — run previous combine cell first.")

df = pd.read_csv(inp, low_memory=False)
smiles = df['SMILES_can'].astype(str).tolist()

print("Computing Morgan fingerprints...")
fps = []
for s in tqdm(smiles, desc="FPs"):
    m = Chem.MolFromSmiles(s)
    if m is None:
        fps.append(None)
    else:
        fps.append(AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048))

n = len(fps)
keep = []
seen = set()
for i in tqdm(range(n), desc="Deduping by Tanimoto"):
    if i in seen:
        continue
    fi = fps[i]

    keep.append(i)
    if fi is None:
        continue
    for j in range(i+1, n):
        if j in seen:
            continue
        fj = fps[j]
        if fj is None:
            continue
        if DataStructs.TanimotoSimilarity(fi, fj) > 0.95:
            seen.add(j)

df_dedupe = df.iloc[keep].reset_index(drop=True)
outpath = os.path.join(WORKDIR, "moltox_dedup_t95.csv")
df_dedupe.to_csv(outpath, index=False)
print("Deduped: from", len(df), "->", len(df_dedupe), "rows. Saved to:", outpath)
df_dedupe.head()


Computing Morgan fingerprints...


Deduping by Tanimoto: 100%|██████████| 4518/4518 [00:22<00:00, 202.73it/s] 


Deduped: from 4518 -> 4062 rows. Saved to: /content/drive/MyDrive/moltox_project/moltox_dedup_t95.csv


,SMILES_can,label
0,O,0
1,C1=CC(=C[N+](=C1)C2C(C(C(O2)COP(=O)(O)OP(=O)(O...,0
2,O=O,0
3,C(=O)=O,0
4,CC1=CC2=C(C=C1C)N(C3=NC(=O)NC(=O)C3=N2)CC(C(C(...,0


In [ ]:

import os, pandas as pd, numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit.Chem import MACCSkeys
from rdkit import RDLogger
from tqdm import tqdm
RDLogger.DisableLog('rdApp.*')

WORKDIR = os.getenv("WORKDIR", "/content/drive/MyDrive/moltox_project")
inp = os.path.join(WORKDIR, "moltox_dedup_t95.csv")
if not os.path.exists(inp):
    raise FileNotFoundError(f"{inp} not found — run previous cell first.")

df = pd.read_csv(inp, low_memory=False)
smiles = df['SMILES_can'].astype(str).tolist()


desc_list = Descriptors._descList
desc_names = [name for name, _ in desc_list]
print("Computing", len(desc_names), "RDKit descriptors...")

desc_vals = []
mols = []
for s in tqdm(smiles, desc="Descriptors"):
    m = Chem.MolFromSmiles(s)
    mols.append(m)
    vals = []
    for name, func in desc_list:
        try:
            vals.append(func(m))
        except Exception:
            vals.append(np.nan)
    desc_vals.append(vals)
df_desc = pd.DataFrame(desc_vals, columns=desc_names)


print("Computing fingerprints (MACCS + Morgan 2048)...")
fps_morgan = []
fps_maccs = []
for m in tqdm(mols, desc="Fingerprints"):
    if m is None:
        fps_morgan.append(None)
        fps_maccs.append(None)
        continue
    fps_morgan.append(AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048))
    try:
        fps_maccs.append(MACCSkeys.GenMACCSKeys(m))
    except Exception:
        fps_maccs.append(None)
y
from rdkit import DataStructs
def bitvect_to_numpy(bv):
    if bv is None:
        return np.zeros((bv.GetNumBits() if bv is not None else 2048,), dtype=int)
    arr = np.zeros((bv.GetNumBits(),), dtype=int)
    DataStructs.ConvertToNumpyArray(bv, arr)
    return arr

X_morgan = np.vstack([bitvect_to_numpy(x) for x in fps_morgan])

maccs_len = 167
X_maccs = np.vstack([bitvect_to_numpy(x)[:maccs_len] if x is not None else np.zeros((maccs_len,), dtype=int) for x in fps_maccs])


df_fp_morgan = pd.DataFrame(X_morgan, columns=[f"morg_{i}" for i in range(X_morgan.shape[1])])
df_fp_maccs = pd.DataFrame(X_maccs, columns=[f"maccs_{i}" for i in range(X_maccs.shape[1])])


X = pd.concat([df_desc.reset_index(drop=True), df_fp_maccs.reset_index(drop=True), df_fp_morgan.reset_index(drop=True)], axis=1)
y = df['label'].values


X.to_parquet(os.path.join(WORKDIR, "features_raw.parquet"), index=False)
pd.DataFrame({'SMILES_can': df['SMILES_can'], 'label': df['label']}).to_csv(os.path.join(WORKDIR, "smiles_labels.csv"), index=False)
print("Saved features_raw.parquet and smiles_labels.csv. Feature shape:", X.shape)


Computing 217 RDKit descriptors...


Descriptors: 100%|██████████| 4062/4062 [00:40<00:00, 101.13it/s]


Computing fingerprints (MACCS + Morgan 2048)...


Fingerprints: 100%|██████████| 4062/4062 [00:02<00:00, 1586.36it/s]


Saved features_raw.parquet and smiles_labels.csv. Feature shape: (4062, 2432)


In [ ]:
import pandas as pd, os

WORKDIR = "/content/drive/MyDrive/moltox_project"
df_check = pd.read_csv(os.path.join(WORKDIR, "moltox_dedup_t95.csv"))
print("Shape:", df_check.shape)
print("Columns:", df_check.columns.tolist())
print(df_check.head(20))


Shape: (4062, 2)
Columns: ['SMILES_can', 'label']
                                           SMILES_can  label
0                                                   O      0
1   C1=CC(=C[N+](=C1)C2C(C(C(O2)COP(=O)(O)OP(=O)(O...      0
2                                                 O=O      0
3                                             C(=O)=O      0
4   CC1=CC2=C(C=C1C)N(C3=NC(=O)NC(=O)C3=N2)CC(C(C(...      0
5                                C(CC(=O)O)C(C(=O)O)N      0
6                                                  OO      0
7                                             CC(=O)O      0
8                                          C(C(=O)O)N      0
9                                         CC(C(=O)O)N      0
10                                C(C(C(=O)O)N)C(=O)O      0
11             C(CC(=O)NC(CS)C(=O)NCC(=O)O)C(C(=O)O)N      0
12                               C(CC(=O)N)C(C(=O)O)N      0
13                                     C(C(C(=O)O)N)O      0
14                                 

In [ ]:
from rdkit import Chem

smiles = df_check['SMILES_can'].dropna().astype(str).tolist()
desc_data, fp_data, bad_smiles = [], [], []

for i, s in enumerate(smiles):
    m = Chem.MolFromSmiles(s)
    if m is None:
        bad_smiles.append(s)
        continue
    try:

        Chem.MolToSmiles(m)
    except Exception as e:
        bad_smiles.append(s)

print("Total input:", len(smiles))
print("Parsed ok:", len(smiles)-len(bad_smiles))
print("Bad:", len(bad_smiles))
print("Sample bad:", bad_smiles[:20])


Total input: 4062
Parsed ok: 4062
Bad: 0
Sample bad: []


In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, rdMolDescriptors
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.AtomPairs import Pairs
import numpy as np

s = "CC(=O)O"
m = Chem.MolFromSmiles(s)


desc_vals = [func(m) for name, func in Descriptors.descList]
print("Num descriptors:", len(desc_vals))


fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(m, 2, nBits=2048)
arr = np.zeros((2048,), dtype=int)
DataStructs.ConvertToNumpyArray(fp, arr)
print("Morgan bits set:", arr.sum())


av = pyAvalonTools.GetAvalonFP(m, 256)
arr2 = np.zeros((256,), dtype=int)
DataStructs.ConvertToNumpyArray(av, arr2)
print("Avalon bits set:", arr2.sum())

from rdkit.Chem.AtomPairs import Pairs, Torsions


ap = rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(m, nBits=512)
arr3 = np.zeros((512,), dtype=int)
DataStructs.ConvertToNumpyArray(ap, arr3)
print("AtomPairs bits set:", arr3.sum())



Num descriptors: 217
Morgan bits set: 7
Avalon bits set: 10
AtomPairs bits set: 6


In [ ]:

import os, pandas as pd, numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, rdMolDescriptors
from rdkit.Avalon import pyAvalonTools
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from joblib import dump

WORKDIR = "/content/drive/MyDrive/moltox_project"

df = pd.read_csv(os.path.join(WORKDIR, "moltox_dedup_t95.csv"))
print("Input dataset:", df.shape)

smiles = df['SMILES_can'].dropna().astype(str).tolist()
labels = df['label'].values

desc_data, fp_data, bad_smiles = [], [], []

for s in smiles:
    m = Chem.MolFromSmiles(s)
    if m is None:
        bad_smiles.append(s)
        continue
    try:

        desc_vals = [func(m) for name, func in Descriptors.descList]
        desc_data.append(desc_vals)

        fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(m, 2, nBits=2048)
        arr = np.zeros((2048,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, arr)


        av = pyAvalonTools.GetAvalonFP(m, 256)
        arr2 = np.zeros((256,), dtype=int)
        DataStructs.ConvertToNumpyArray(av, arr2)


        ap = rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(m, nBits=512)
        arr3 = np.zeros((512,), dtype=int)
        DataStructs.ConvertToNumpyArray(ap, arr3)


        fp_all = np.concatenate([arr, arr2, arr3])
        fp_data.append(fp_all)

    except Exception:
        bad_smiles.append(s)

print("Descriptors extracted:", len(desc_data))
print("Fingerprints extracted:", len(fp_data))
print("Bad SMILES skipped:", len(bad_smiles))


desc_cols = [name for name, func in Descriptors.descList]
df_desc = pd.DataFrame(desc_data, columns=desc_cols)
df_fp = pd.DataFrame(fp_data)

X = pd.concat([df_desc, df_fp], axis=1)
print("Feature matrix shape:", X.shape)


X = X.replace([np.inf, -np.inf], np.nan)
X = X.mask(X.abs() > 1e6)


imp = SimpleImputer(strategy='median')
X_desc_imp = pd.DataFrame(imp.fit_transform(X[desc_cols]), columns=desc_cols)


mi = mutual_info_classif(X_desc_imp, labels, discrete_features=False, random_state=42)
mi_sel = pd.Series(mi, index=desc_cols).sort_values(ascending=False)
top_desc = mi_sel.index[:200].tolist()

print("Top descriptors:", top_desc[:10])

X_final = pd.concat(
    [X_desc_imp[top_desc], X.drop(columns=desc_cols).reset_index(drop=True)],
    axis=1
)


X_final.columns = X_final.columns.astype(str)
print("Final feature matrix:", X_final.shape)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)


np.save(os.path.join(WORKDIR, "X_scaled.npy"), X_scaled)
np.save(os.path.join(WORKDIR, "y.npy"), labels)
dump(scaler, os.path.join(WORKDIR, "scaler.joblib"))

print("Saved X_scaled.npy, y.npy, and scaler.joblib")



Input dataset: (4062, 2)
Descriptors extracted: 4062
Fingerprints extracted: 4062
Bad SMILES skipped: 0
Feature matrix shape: (4062, 3033)
Top descriptors: ['MaxAbsPartialCharge', 'MinPartialCharge', 'BCUT2D_MWHI', 'MaxPartialCharge', 'MinAbsPartialCharge', 'BalabanJ', 'SlogP_VSA2', 'TPSA', 'PEOE_VSA1', 'BCUT2D_MRHI']
Final feature matrix: (4062, 3016)
Saved X_scaled.npy, y.npy, and scaler.joblib


In [ ]:
# # Diagnostic: check intermediate files, labels and show sample SMILES
# import os, pandas as pd, numpy as np
# WORKDIR = "/content/drive/MyDrive/moltox_project"
# print("WORKDIR:", WORKDIR)
# for fname in ["moltox_raw_combined.csv", "moltox_dedup_t95.csv", "smiles_labels.csv", "t3db_canon_from_csv.csv", "t3db_canon_from_sdf.csv", "kegg_mapped_pubchem.csv", "tox_pubchem_smiles.csv"]:
#     path = os.path.join(WORKDIR, fname)
#     exists = os.path.exists(path)
#     print(f"\n{fname}: exists={exists}")
#     if exists:
#         try:
#             df = pd.read_csv(path, nrows=20, low_memory=False)
#             print("  shape:", pd.read_csv(path).shape if fname in ["moltox_raw_combined.csv","moltox_dedup_t95.csv","smiles_labels.csv"] else df.shape)
#             print("  columns:", df.columns.tolist())
#             if 'label' in df.columns:
#                 print("  label value counts:\n", pd.read_csv(path)['label'].value_counts(dropna=False).to_dict())
#             # show first 10 SMILES-like values if present
#             for col in ['SMILES_can','SMILES','smiles','SMILES_CAN']:
#                 if col in df.columns:
#                     print("  sample values from", col, ":", df[col].dropna().astype(str).unique()[:10].tolist())
#                     break
#         except Exception as e:
#             print("  could not preview file:", e)

# # Also check the loaded numpy arrays used by training
# xpath = os.path.join(WORKDIR, "X_scaled.npy")
# ypath = os.path.join(WORKDIR, "y.npy")
# print("\nX_scaled.npy exists:", os.path.exists(xpath))
# print("y.npy exists:", os.path.exists(ypath))
# if os.path.exists(ypath):
#     y = np.load(ypath)
#     unique, counts = np.unique(y, return_counts=True)
#     print("y.npy unique:", dict(zip(unique, counts)))


In [ ]:

import os, numpy as np, pandas as pd, joblib, warnings, time
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, matthews_corrcoef
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

WORKDIR = os.getenv("WORKDIR", "/content/drive/MyDrive/moltox_project")
X = np.load(os.path.join(WORKDIR, "X_scaled.npy"))
y = np.load(os.path.join(WORKDIR, "y.npy"))

print("Loaded X shape:", X.shape, "y shape:", y.shape)
pos_count = int((y==1).sum()); neg_count = int((y==0).sum())
print("Dataset class counts -> positive:", pos_count, "negative:", neg_count)


if pos_count == 0 or neg_count == 0:
    raise ValueError("Dataset contains only one class overall. Add examples for the missing class before training.")

def get_positive_proba(est, X_in):
    if hasattr(est, "predict_proba"):
        probs = est.predict_proba(X_in)
        if probs.ndim == 2 and probs.shape[1] >= 2:
            if hasattr(est, "classes_"):
                classes = list(est.classes_)
                if 1 in classes:
                    idx = classes.index(1)
                    return probs[:, idx]
            return probs[:, 1]
        if probs.ndim == 2 and probs.shape[1] == 1:

            if hasattr(est, "classes_") and len(est.classes_) == 1:
                if est.classes_[0] == 1:
                    return probs[:, 0]
                else:
                    return 1.0 - probs[:, 0]
            else:
                return np.zeros((probs.shape[0],), dtype=float)
    try:
        preds = est.predict(X_in)
        return (preds == 1).astype(float)
    except Exception:
        return np.zeros((X_in.shape[0],), dtype=float)

rf = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_params = {'n_estimators':[100,200], 'max_depth':[None,10,30], 'min_samples_leaf':[1,4,10]}

lgb = LGBMClassifier(class_weight='balanced', random_state=42, n_jobs=-1)
lgb_params = {'num_leaves':[31,63,127], 'learning_rate':[0.05,0.1], 'n_estimators':[100,200]}

mlp = MLPClassifier(random_state=42, max_iter=300)
mlp_params = {'hidden_layer_sizes':[(128,64),(256,128)], 'alpha':[1e-4,1e-3], 'learning_rate_init':[1e-3,1e-4]}


cv_small = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
def quick_tune(clf, params, Xt, yt, n_iter=6):
    rs = RandomizedSearchCV(clf, params, n_iter=n_iter, cv=cv_small, scoring='roc_auc', n_jobs=1, random_state=42)
    rs.fit(Xt, yt)
    print(f"Best {type(clf).__name__} AUC {rs.best_score_:.4f}")
    return rs.best_estimator_


valid_split = False
max_attempts = 12
for seed in range(42, 42 + max_attempts):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=seed)
    if len(np.unique(y_train)) >= 2 and len(np.unique(y_test)) >= 2:
        print(f"Using stratified split with random_state={seed}")
        valid_split = True
        break
    else:
        print(f"Attempt {seed-41}: split invalid (train classes {np.unique(y_train)}, test classes {np.unique(y_test)})")
if not valid_split:
    warnings.warn("Could not find a stratified train/test split with both classes present in train and test. "
                  "Falling back to full cross-validated stacking (no separate holdout test).")


if valid_split:
    pos_train = int((y_train==1).sum())
    requested_folds = 5
    n_folds = min(requested_folds, max(2, pos_train))
    if n_folds < requested_folds:
        warnings.warn(f"Reduced n_folds to {n_folds} because positive count in train is {pos_train}.")

    print("Tuning base learners on training set...")
    best_rf = quick_tune(rf, rf_params, X_train, y_train, n_iter=6)
    best_lgb = quick_tune(lgb, lgb_params, X_train, y_train, n_iter=6)
    best_mlp = quick_tune(mlp, mlp_params, X_train, y_train, n_iter=6)


    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    meta_train = np.zeros((X_train.shape[0], 3))
    meta_test_blend = np.zeros((X_test.shape[0], 3))

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        print(f"Fold {fold+1}/{n_folds}")
        X_tr, X_val = X_train[tr_idx], X_train[val_idx]
        y_tr, y_val = y_train[tr_idx], y_train[val_idx]

        if len(np.unique(y_tr)) == 1:
            warnings.warn(f"Fold {fold+1} train has only class {np.unique(y_tr)[0]}")

        est_rf = best_rf.__class__(**best_rf.get_params())
        est_lgb = best_lgb.__class__(**best_lgb.get_params())
        est_mlp = best_mlp.__class__(**best_mlp.get_params())

        est_rf.fit(X_tr, y_tr)
        est_lgb.fit(X_tr, y_tr)
        est_mlp.fit(X_tr, y_tr)

        meta_train[val_idx, 0] = get_positive_proba(est_rf, X_val)
        meta_train[val_idx, 1] = get_positive_proba(est_lgb, X_val)
        meta_train[val_idx, 2] = get_positive_proba(est_mlp, X_val)

        meta_test_blend[:, 0] += get_positive_proba(est_rf, X_test) / n_folds
        meta_test_blend[:, 1] += get_positive_proba(est_lgb, X_test) / n_folds
        meta_test_blend[:, 2] += get_positive_proba(est_mlp, X_test) / n_folds


    if len(np.unique(y_train)) < 2:
        raise ValueError("After OOF creation y_train contains only one class — cannot train meta learner.")
    meta = LogisticRegression(class_weight='balanced', solver='lbfgs', max_iter=500)
    meta.fit(meta_train, y_train)


    y_test_prob = meta.predict_proba(meta_test_blend)[:, 1]
    y_test_pred = (y_test_prob >= 0.5).astype(int)

    auc = roc_auc_score(y_test, y_test_prob)
    acc = accuracy_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp+fn)>0 else 0.0
    specificity = tn / (tn + fp) if (tn+fp)>0 else 0.0
    mcc = matthews_corrcoef(y_test, y_test_pred)

    print("Holdout test results -> AUC: {:.4f}, Acc: {:.4f}, F1: {:.4f}, Sens: {:.4f}, Spec: {:.4f}, MCC: {:.4f}".format(
        auc, acc, f1, sensitivity, specificity, mcc))


    pd.DataFrame(meta_train, columns=['rf_prob','lgbm_prob','mlp_prob']).to_csv(os.path.join(WORKDIR, "meta_train_oof.csv"), index=False)
    joblib.dump(best_rf, os.path.join(WORKDIR, "best_rf.joblib"))
    joblib.dump(best_lgb, os.path.join(WORKDIR, "best_lgb.joblib"))
    joblib.dump(best_mlp, os.path.join(WORKDIR, "best_mlp.joblib"))
    joblib.dump(meta, os.path.join(WORKDIR, "meta_logreg.joblib"))
    print("Saved models and OOF preds.")

else:

    print("Fallback: performing full cross-validated stacking (OOF on whole dataset).")
    pos_total = int((y==1).sum())
    n_folds = min(5, max(2, pos_total))
    if n_folds < 3:
        warnings.warn(f"Very few positives ({pos_total}); using n_folds={n_folds}.")

   )
    best_rf = quick_tune(rf, rf_params, X, y, n_iter=6)
    best_lgb = quick_tune(lgb, lgb_params, X, y, n_iter=6)
    best_mlp = quick_tune(mlp, mlp_params, X, y, n_iter=6)

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    meta_oof = np.zeros((X.shape[0], 3))

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"Fold {fold+1}/{n_folds}")
        X_tr, X_val = X[tr_idx], X[val_idx]
        y_tr, y_val = y[tr_idx], y[val_idx]

        est_rf = best_rf.__class__(**best_rf.get_params())
        est_lgb = best_lgb.__class__(**best_lgb.get_params())
        est_mlp = best_mlp.__class__(**best_mlp.get_params())

        est_rf.fit(X_tr, y_tr)
        est_lgb.fit(X_tr, y_tr)
        est_mlp.fit(X_tr, y_tr)

        meta_oof[val_idx, 0] = get_positive_proba(est_rf, X_val)
        meta_oof[val_idx, 1] = get_positive_proba(est_lgb, X_val)
        meta_oof[val_idx, 2] = get_positive_proba(est_mlp, X_val)


    if len(np.unique(y)) < 2:
        raise ValueError("Cannot train meta: dataset contains only one class.")
    meta = LogisticRegression(class_weight='balanced', solver='lbfgs', max_iter=500)
    meta.fit(meta_oof, y)

   a)
    try:
        auc = roc_auc_score(y, meta.predict_proba(meta_oof)[:,1])
    except Exception:
        auc = float('nan')
    preds = (meta.predict_proba(meta_oof)[:,1] >= 0.5).astype(int)
    acc = accuracy_score(y, preds)
    f1 = f1_score(y, preds)
    tn, fp, fn, tp = confusion_matrix(y, preds).ravel()
    sensitivity = tp / (tp + fn) if (tp+fn)>0 else 0.0
    specificity = tn / (tn + fp) if (tn+fp)>0 else 0.0
    mcc = matthews_corrcoef(y, preds)

    print("CV-stacking results (OOF) -> AUC: {:.4f}, Acc: {:.4f}, F1: {:.4f}, Sens: {:.4f}, Spec: {:.4f}, MCC: {:.4f}".format(
        auc, acc, f1, sensitivity, specificity, mcc))

    # save artifacts
    pd.DataFrame(meta_oof, columns=['rf_prob','lgbm_prob','mlp_prob']).to_csv(os.path.join(WORKDIR, "meta_oof_full.csv"), index=False)
    joblib.dump(best_rf, os.path.join(WORKDIR, "best_rf.joblib"))
    joblib.dump(best_lgb, os.path.join(WORKDIR, "best_lgb.joblib"))
    joblib.dump(best_mlp, os.path.join(WORKDIR, "best_mlp.joblib"))
    joblib.dump(meta, os.path.join(WORKDIR, "meta_logreg.joblib"))
    print("Saved models and OOF preds (full CV fallback).")


Loaded X shape: (4062, 3016) y shape: (4062,)
Dataset class counts -> positive: 2465 negative: 1597
Using stratified split with random_state=42
Tuning base learners on training set...
Best RandomForestClassifier AUC 0.9170
[LightGBM] [Info] Number of positive: 1314, number of negative: 852
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28455
[LightGBM] [Info] Number of data points in the train set: 2166, number of used features: 1773
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 1315, number of negative: 851
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021916 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28263
[LightGBM] [Info] Number of data points in the train set: 2166, number of used features: 1734
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 1315, number of negative: 851
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023533 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28164
[LightGBM] [Info] Number of data points in the train set: 2166, number of used features: 1757
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


...

Fold 4/5
[LightGBM] [Info] Number of positive: 1578, number of negative: 1021
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.234056 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29205
[LightGBM] [Info] Number of data points in the train set: 2599, number of used features: 1937
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Fold 5/5
[LightGBM] [Info] Number of positive: 1578, number of negative: 1022
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.189108 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 29152
[LightGBM] [Info] Number of data points in the train set: 2600, number of used features: 1933
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Holdout test results -> AUC: 0.9422, Acc: 0.8831, F1: 0.9016, Sens: 0.8824, Spec: 0.8844, MCC: 0.7591
Saved models and OOF preds.


In [ ]:
import os, joblib, numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

WORKDIR = "/content/drive/MyDrive/moltox_project"
X = np.load(os.path.join(WORKDIR, "X_scaled.npy"))
y = np.load(os.path.join(WORKDIR, "y.npy"))
n = X.shape[0]
print("X,y shapes:", X.shape, y.shape)
N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

base_defs = [
    ("rf", RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)),
    ("lgbm", LGBMClassifier(n_estimators=200, class_weight='balanced', random_state=42)),
    ("mlp", MLPClassifier(hidden_layer_sizes=(256,128), alpha=1e-4, max_iter=300, random_state=42))
]

meta_oof = pd.DataFrame(index=range(n))

for name, model in base_defs:
    print("Generating OOF for:", name)
    oof = np.zeros(n)
    for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
        print(" Fold", fold+1, "train", len(tr_idx), "val", len(val_idx))
        m = model.__class__(**model.get_params())
        m.fit(X[tr_idx], y[tr_idx])
        if hasattr(m, "predict_proba"):
            probs = m.predict_proba(X[val_idx])
            if probs.shape[1] == 1:
                probs1 = probs[:,0] if (hasattr(m, "classes_") and m.classes_[0]==1) else 1-probs[:,0]
            else:
                probs1 = probs[:, list(m.classes_).index(1)] if 1 in m.classes_ else probs[:,1]
        else:
            probs1 = m.predict(X[val_idx]).astype(float)
        oof[val_idx] = probs1
    meta_oof[name + "_prob"] = oof

    final_m = model.__class__(**model.get_params())
    final_m.fit(X, y)
    joblib.dump(final_m, os.path.join(WORKDIR, f"retrained_full_{name}.joblib"))
    print("Saved full-data refit for", name)

out_true = os.path.join(WORKDIR, "meta_oof_true.csv")
meta_oof.to_csv(out_true, index=False)
print("Saved true OOF to", out_true, "shape:", meta_oof.shape)


X,y shapes: (4062, 3016) (4062,)
Generating OOF for: rf
 Fold 1 train 3249 val 813
 Fold 2 train 3249 val 813
 Fold 3 train 3250 val 812
 Fold 4 train 3250 val 812
 Fold 5 train 3250 val 812
Saved full-data refit for rf
Generating OOF for: lgbm
 Fold 1 train 3249 val 813
[LightGBM] [Info] Number of positive: 1972, number of negative: 1277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044415 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30458
[LightGBM] [Info] Number of data points in the train set: 3249, number of used features: 2201
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


 Fold 2 train 3249 val 813
[LightGBM] [Info] Number of positive: 1972, number of negative: 1277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.051967 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30361
[LightGBM] [Info] Number of data points in the train set: 3249, number of used features: 2183
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


 Fold 3 train 3250 val 812
[LightGBM] [Info] Number of positive: 1972, number of negative: 1278
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050314 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30227
[LightGBM] [Info] Number of data points in the train set: 3250, number of used features: 2163
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


...

 Fold 4 train 3250 val 812
[LightGBM] [Info] Number of positive: 1972, number of negative: 1278
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.079388 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30287
[LightGBM] [Info] Number of data points in the train set: 3250, number of used features: 2163
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


 Fold 5 train 3250 val 812
[LightGBM] [Info] Number of positive: 1972, number of negative: 1278
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.043111 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30162
[LightGBM] [Info] Number of data points in the train set: 3250, number of used features: 2161
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 2465, number of negative: 1597
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.127700 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31314
[LightGBM] [Info] Number of data points in the train set: 4062, number of used features: 2395
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Saved full-data refit for lgbm
Generating OOF for: mlp
 Fold 1 train 3249 val 813
 Fold 2 train 3249 val 813
 Fold 3 train 3250 val 812
 Fold 4 train 3250 val 812
 Fold 5 train 3250 val 812
Saved full-data refit for mlp
Saved true OOF to /content/drive/MyDrive/moltox_project/meta_oof_true.csv shape: (4062, 3)


In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, confusion_matrix, matthews_corrcoef, brier_score_loss
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold

WORKDIR = "/content/drive/MyDrive/moltox_project"
meta_path = os.path.join(WORKDIR, "meta_oof_true.csv")
y_path = os.path.join(WORKDIR, "y.npy")

meta = pd.read_csv(meta_path)
y = np.load(y_path)
print("Meta shape:", meta.shape, "y shape:", y.shape)

candidates = {
    "LogReg(L2)": LogisticRegression(class_weight="balanced", max_iter=500, solver="lbfgs"),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "NaiveBayes": GaussianNB(),
    "MLP_small": MLPClassifier(hidden_layer_sizes=(32,), max_iter=300, random_state=42),
    "RF_meta": RandomForestClassifier(n_estimators=200, class_weight="balanced", random_state=42),
    "GB_meta": GradientBoostingClassifier(n_estimators=200, random_state=42),
}

results = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, clf in candidates.items():
    aucs, accs, f1s, sens, specs, mccs, briers = [], [], [], [], [], [], []
    for tr, val in skf.split(meta, y):
        clf.fit(meta.iloc[tr], y[tr])
        preds = clf.predict(meta.iloc[val])
        probs = clf.predict_proba(meta.iloc[val])[:,1] if hasattr(clf, "predict_proba") else preds.astype(float)

        aucs.append(roc_auc_score(y[val], probs))
        accs.append(accuracy_score(y[val], preds))
        f1s.append(f1_score(y[val], preds))
        rec = recall_score(y[val], preds)
        sens.append(rec)
        tn, fp, fn, tp = confusion_matrix(y[val], preds).ravel()
        specs.append(tn/(tn+fp))
        mccs.append(matthews_corrcoef(y[val], preds))
        briers.append(brier_score_loss(y[val], probs))

    results.append({
        "name": name,
        "AUC": np.mean(aucs),
        "ACC": np.mean(accs),
        "F1": np.mean(f1s),
        "Sens": np.mean(sens),
        "Spec": np.mean(specs),
        "MCC": np.mean(mccs),
        "Brier": np.mean(briers)
    })

dfres = pd.DataFrame(results).sort_values(by=["AUC","F1"], ascending=False).reset_index(drop=True)
print(dfres)


Meta shape: (4062, 3) y shape: (4062,)
         name       AUC       ACC        F1      Sens      Spec       MCC  \
0         LDA  0.946816  0.877154  0.898847  0.899797  0.842230  0.742865   
1         QDA  0.946526  0.879862  0.900369  0.894929  0.856634  0.749662   
2     GB_meta  0.946307  0.872480  0.895718  0.902231  0.826589  0.732211   
3   MLP_small  0.945257  0.880107  0.901541  0.905071  0.841607  0.748818   
4  LogReg(L2)  0.944987  0.877894  0.898160  0.887627  0.862900  0.746527   
5  NaiveBayes  0.943146  0.872476  0.893320  0.880325  0.860388  0.735821   
6     RF_meta  0.942328  0.871247  0.894788  0.902231  0.823450  0.729631   

      Brier  
0  0.100717  
1  0.101201  
2  0.090962  
3  0.090205  
4  0.092464  
5  0.113358  
6  0.092339  


In [ ]:
!pip install --quiet deepchem rdkit

import os, time
import numpy as np, pandas as pd
from rdkit import Chem
from deepchem.feat import MolGraphConvFeaturizer

WORKDIR = "/content/drive/MyDrive/moltox_project"
SMILES_CSV = os.path.join(WORKDIR, "smiles_labels.csv")
Y_PATH = os.path.join(WORKDIR, "y.npy")
FAILED_CSV = os.path.join(WORKDIR, "gcn_failed_smiles_one_by_one.csv")
FEAT_SAVE = os.path.join(WORKDIR, "gcn_valid_index_map.npy")

df = pd.read_csv(SMILES_CSV)
smiles_all = df['SMILES_can'].astype(str).tolist()
y = np.load(Y_PATH)
assert len(smiles_all) == len(y)

valid_idx = []
invalid_idx = []
for i, s in enumerate(smiles_all):
    try:
        mol = Chem.MolFromSmiles(s)
        if mol is None:
            invalid_idx.append(i)
        else:
            try:
                Chem.SanitizeMol(mol)
                valid_idx.append(i)
            except Exception:
                invalid_idx.append(i)
    except Exception:
        invalid_idx.append(i)

print(f"Total: {len(smiles_all)}  RDKit-sane: {len(valid_idx)}  invalid: {len(invalid_idx)}")
if invalid_idx:
    pd.DataFrame({"index": invalid_idx, "smiles": [smiles_all[i] for i in invalid_idx]}).to_csv(FAILED_CSV, index=False)
    print("Saved RDKit-invalid SMILES to", FAILED_CSV)

featurizer = MolGraphConvFeaturizer()
valid_graphs = []
valid_graphs_idx = []
failed_after_feat = []

t0 = time.time()
for pos, orig_i in enumerate(valid_idx):
    s = smiles_all[orig_i]
    try:
        g = featurizer.featurize([s])[0]
        if g is None:
            failed_after_feat.append(orig_i)
        else:
            if hasattr(g, 'node_features'):
                if getattr(g, 'node_features') is None or len(getattr(g, 'node_features')) == 0:
                    failed_after_feat.append(orig_i)
                else:
                    valid_graphs.append(g)
                    valid_graphs_idx.append(orig_i)
            else:
                valid_graphs.append(g)
                valid_graphs_idx.append(orig_i)
    except Exception as e:
        failed_after_feat.append(orig_i)
        if len(failed_after_feat) <= 10:
            print(f"Featurize failed at original idx {orig_i} SMILES: {s}  -> {type(e).__name__}: {e}")

print(f"Per-molecule featurization done in {time.time()-t0:.1f}s")
print("Succeeded:", len(valid_graphs), "Failed after RDKit:", len(failed_after_feat))

all_failed = sorted(set(invalid_idx) | set(failed_after_feat))
if all_failed:
    pd.DataFrame({"index": all_failed, "smiles": [smiles_all[i] for i in all_failed]}).to_csv(FAILED_CSV, index=False)
    print("Saved all failed SMILES to", FAILED_CSV)

np.save(FEAT_SAVE, {"valid_idx": valid_graphs_idx})
print("Saved mapping of valid indices to", FEAT_SAVE)

aligned_oof = np.full((len(smiles_all),), np.nan, dtype=float)
print("Ready. Example: aligned_oof shape", aligned_oof.shape)


Total: 4062  RDKit-sane: 4062  invalid: 0


Per-molecule featurization done in 28.3s
Succeeded: 4062 Failed after RDKit: 0
Saved mapping of valid indices to /content/drive/MyDrive/moltox_project/gcn_valid_index_map.npy
Ready. Example: aligned_oof shape (4062,)


In [ ]:
!pip install --quiet deepchem rdkit-pypi

import os, time, pickle
import numpy as np, pandas as pd
from rdkit import Chem

from deepchem.feat import MolGraphConvFeaturizer, ConvMolFeaturizer
from deepchem.data import NumpyDataset

WORKDIR = "/content/drive/MyDrive/moltox_project"
SMILES_CSV = os.path.join(WORKDIR, "smiles_labels.csv")
Y_PATH = os.path.join(WORKDIR, "y.npy")
OUT_FAILED = os.path.join(WORKDIR, "gcn_failed_smiles_debug.csv")
OUT_GOOD_PKL = os.path.join(WORKDIR, "gcn_valid_graphs.pkl")
OUT_IDX = os.path.join(WORKDIR, "gcn_valid_idx.npy")

# load
df = pd.read_csv(SMILES_CSV)
smiles_all = df['SMILES_can'].astype(str).tolist()
y = np.load(Y_PATH)
assert len(smiles_all) == len(y)

# instantiate featurizers
mg_featurizer = MolGraphConvFeaturizer()
conv_featurizer = ConvMolFeaturizer()

good_graphs = []
good_idx = []
failed_records = []

t0 = time.time()
for i, s in enumerate(smiles_all):
    try:
        mol = Chem.MolFromSmiles(s)
        if mol is None:
            failed_records.append({"index": i, "smiles": s, "stage": "rdkit_parse", "error": "RDKit MolFromSmiles returned None"})
            continue
        try:
            Chem.SanitizeMol(mol)
        except Exception as e:
            failed_records.append({"index": i, "smiles": s, "stage": "rdkit_sanitize", "error": repr(e)})
    except Exception as e:
        failed_records.append({"index": i, "smiles": s, "stage": "rdkit_exception", "error": repr(e)})
        continue

    try:
        g = mg_featurizer.featurize([s])[0]
        if g is None:
            raise ValueError("MolGraphConvFeaturizer returned None")
        if hasattr(g, "node_features") and (getattr(g, "node_features") is None or len(getattr(g, "node_features")) == 0):
            raise ValueError("MolGraphConvFeaturizer returned empty node_features")
        # success
        good_graphs.append(g)
        good_idx.append(i)
        continue
    except Exception as e_mg:
        mg_err = repr(e_mg)
        try:
            g2 = conv_featurizer.featurize([s])[0]
            if g2 is None:
                raise ValueError("ConvMolFeaturizer returned None")
            good_graphs.append(g2)
            good_idx.append(i)
            continue
        except Exception as e_conv:
            conv_err = repr(e_conv)
            failed_records.append({
                "index": i,
                "smiles": s,
                "stage": "featurization_both_failed",
                "error": f"MolGraphErr: {mg_err} | ConvMolErr: {conv_err}"
            })
            continue

elapsed = time.time() - t0
print(f"Featurization loop done in {elapsed:.1f}s")
print("Good graphs:", len(good_graphs), "Failed entries:", len(failed_records))

if failed_records:
    pd.DataFrame(failed_records).to_csv(OUT_FAILED, index=False)
    print("Saved failure debug CSV to:", OUT_FAILED)

with open(OUT_GOOD_PKL, "wb") as f:
    pickle.dump(good_graphs, f)
np.save(OUT_IDX, np.array(good_idx, dtype=int))
print("Saved valid graphs pickle to:", OUT_GOOD_PKL)
print("Saved valid index mapping to:", OUT_IDX)

ERROR: Could not find a version that satisfies the requirement rdkit-pypi (from versions: none)
ERROR: No matching distribution found for rdkit-pypi


Featurization loop done in 30.3s
Good graphs: 4062 Failed entries: 0
Saved valid graphs pickle to: /content/drive/MyDrive/moltox_project/gcn_valid_graphs.pkl
Saved valid index mapping to: /content/drive/MyDrive/moltox_project/gcn_valid_idx.npy


In [ ]:


import os, time, pickle, traceback
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold

WORKDIR = "/content/drive/MyDrive/moltox_project"
GOOD_PKL = os.path.join(WORKDIR, "gcn_valid_graphs.pkl")
IDX_NPY  = os.path.join(WORKDIR, "gcn_valid_idx.npy")
Y_PATH   = os.path.join(WORKDIR, "y.npy")
OUT_OOF  = os.path.join(WORKDIR, "gcn_oof_aligned_cpu.csv")
GCN_DIR  = os.path.join(WORKDIR, "gcn_models")
LOGFILE  = os.path.join(WORKDIR, "gcn_oof_error_cpu.log")
os.makedirs(GCN_DIR, exist_ok=True)

N_FOLDS    = 3
EPOCHS     = 6
BATCH_SIZE = 8
LEARNING_RATE = 1e-3
SEED = 42

def info(msg): print("[INFO]", msg)

info("Loading pickled graph objects and index mapping...")
with open(GOOD_PKL, "rb") as f:
    valid_graphs = pickle.load(f)
valid_idx = np.load(IDX_NPY).tolist()
y = np.load(Y_PATH)
n_total = len(y)
info(f"Total rows: {n_total}, valid graphs: {len(valid_graphs)}")

if len(valid_graphs) != len(valid_idx):
    raise RuntimeError("valid_graphs and valid_idx length mismatch - re-create mapping.")

gcn_oof_full = np.full((n_total,), np.nan, dtype=float)

fold_done = []
for f in os.listdir(WORKDIR):
    if f.startswith("gcn_oof_partial_fold_") and f.endswith(".npy"):
        try:
            idx = int(f.split("_")[-1].split(".")[0])
            fold_done.append(idx)
        except:
            pass
fold_done = sorted(set(fold_done))
if fold_done:
    info(f"Resuming: found partial fold outputs for folds: {fold_done}")
    for fd in fold_done:
        partial = np.load(os.path.join(WORKDIR, f"gcn_oof_partial_fold_{fd}.npy"), allow_pickle=True).item()
        for orig_i, prob in partial.items():
            gcn_oof_full[int(orig_i)] = float(prob)

valid_y = np.array([ y[i] for i in valid_idx ])

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

from deepchem.data import NumpyDataset
from deepchem.models import GraphConvModel

fold_no = 0
for tr_sub, val_sub in skf.split(np.zeros(len(valid_y)), valid_y):
    fold_no += 1
    if fold_no in fold_done:
        info(f"Skipping fold {fold_no} (already completed).")
        continue

    info(f"Starting fold {fold_no} -> train {len(tr_sub)} val {len(val_sub)}")
    try:
        X_tr = [ valid_graphs[i] for i in tr_sub ]
        y_tr = valid_y[tr_sub].reshape(-1,1)
        X_val = [ valid_graphs[i] for i in val_sub ]
        y_val = valid_y[val_sub].reshape(-1,1)

        ds_tr = NumpyDataset(X_tr, y_tr)
        ds_val = NumpyDataset(X_val, y_val)

        model_dir = os.path.join(GCN_DIR, f"gcn_fold_cpu_{fold_no}")
        model = GraphConvModel(n_tasks=1, mode='classification',
                               batch_size=BATCH_SIZE, model_dir=model_dir,
                               learning_rate=LEARNING_RATE, verbose=0)

        t0 = time.time()
        model.fit(ds_tr, nb_epoch=EPOCHS)
        info(f"Fold {fold_no} trained in {time.time()-t0:.1f}s (epochs={EPOCHS})")

        preds = model.predict(ds_val).reshape(-1)
        info(f"Fold {fold_no} preds min/max: {float(np.min(preds)):.6f}/{float(np.max(preds)):.6f}")

        partial_dict = {}
        for local_i, orig_idx in enumerate([ valid_idx[i] for i in val_sub ]):
            prob = float(preds[local_i])
            gcn_oof_full[orig_idx] = prob
            partial_dict[int(orig_idx)] = prob

        np.save(os.path.join(WORKDIR, f"gcn_oof_partial_fold_{fold_no}.npy"), partial_dict)
        info(f"Saved partial OOF for fold {fold_no} to gcn_oof_partial_fold_{fold_no}.npy")

        # cleanup
        del model

    except Exception as e:
        tb = traceback.format_exc()
        with open(LOGFILE, "a") as lf:
            lf.write(f"=== Fold {fold_no} EXCEPTION ===\n")
            lf.write(tb + "\n\n")
        info(f"Exception in fold {fold_no}; logged to {LOGFILE}. Continuing to next fold.")
        continue

pd.DataFrame({"gcn_prob": gcn_oof_full}).to_csv(OUT_OOF, index=False)
info(f"Saved aligned OOF (CPU) to {OUT_OOF}. NaNs count: {int(np.isnan(gcn_oof_full).sum())} / {n_total}")

try:
    info("Refitting final GCN on entire valid set (low-epoch) and saving to gcn_full_cpu/ ...")
    ds_full = NumpyDataset(valid_graphs, np.array([ y[i] for i in valid_idx ]).reshape(-1,1))
    final_dir = os.path.join(GCN_DIR, "gcn_full_cpu")
    final_model = GraphConvModel(n_tasks=1, mode='classification',
                                 batch_size=BATCH_SIZE, model_dir=final_dir,
                                 learning_rate=LEARNING_RATE, verbose=0)
    final_model.fit(ds_full, nb_epoch=EPOCHS)
    info(f"Final CPU GCN trained and saved at {final_dir}")
except Exception as e:
    tb = traceback.format_exc()
    with open(LOGFILE, "a") as lf:
        lf.write("=== Final refit EXCEPTION ===\n")
        lf.write(tb + "\n\n")
    info("Exception during final refit; logged to " + LOGFILE)

info("CPU run complete. If any folds logged exceptions, open the log file to inspect.")

[INFO] Loading pickled graph objects and index mapping...
[INFO] Total rows: 4062, valid graphs: 4062
[INFO] Starting fold 1 -> train 2708 val 1354
[INFO] Exception in fold 1; logged to /content/drive/MyDrive/moltox_project/gcn_oof_error_cpu.log. Continuing to next fold.
[INFO] Starting fold 2 -> train 2708 val 1354
[INFO] Exception in fold 2; logged to /content/drive/MyDrive/moltox_project/gcn_oof_error_cpu.log. Continuing to next fold.
[INFO] Starting fold 3 -> train 2708 val 1354
[INFO] Exception in fold 3; logged to /content/drive/MyDrive/moltox_project/gcn_oof_error_cpu.log. Continuing to next fold.
[INFO] Saved aligned OOF (CPU) to /content/drive/MyDrive/moltox_project/gcn_oof_aligned_cpu.csv. NaNs count: 4062 / 4062
[INFO] Refitting final GCN on entire valid set (low-epoch) and saving to gcn_full_cpu/ ...
[INFO] Exception during final refit; logged to /content/drive/MyDrive/moltox_project/gcn_oof_error_cpu.log
[INFO] CPU run complete. If any folds logged exceptions, open the log

In [ ]:
import pandas as pd

WORKDIR = "/content/drive/MyDrive/moltox_project"

meta = pd.read_csv(f"{WORKDIR}/meta_oof_true.csv")

gcn = pd.read_csv(f"{WORKDIR}/gcn_oof_aligned_cpu.csv")

print("Meta shape:", meta.shape, "GCN shape:", gcn.shape)

meta["gcn_prob"] = gcn["gcn_prob"]

meta.to_csv(f"{WORKDIR}/meta_oof_true_with_gcn.csv", index=False)
print("Extended meta saved at meta_oof_true_with_gcn.csv")

Meta shape: (4062, 3) GCN shape: (4062, 1)
Extended meta saved at meta_oof_true_with_gcn.csv


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, matthews_corrcoef, brier_score_loss

WORKDIR = "/content/drive/MyDrive/moltox_project"
meta = pd.read_csv(f"{WORKDIR}/meta_oof_true_with_gcn.csv")
y = np.load(f"{WORKDIR}/y.npy")

models = {
    "LogReg(L2)": LogisticRegression(class_weight="balanced", max_iter=500, solver="lbfgs"),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "MLP_small": MLPClassifier(hidden_layer_sizes=(32,), max_iter=300),
    "GB_meta": GradientBoostingClassifier(),
    "RF_meta": RandomForestClassifier(),
    "NaiveBayes": GaussianNB()
}

results = []
for name, clf in models.items():
    try:
        clf.fit(meta, y)
        preds = clf.predict(meta)
        probs = clf.predict_proba(meta)[:,1]

        tn, fp, fn, tp = confusion_matrix(y, preds).ravel()
        results.append({
            "name": name,
            "AUC": roc_auc_score(y, probs),
            "ACC": accuracy_score(y, preds),
            "F1": f1_score(y, preds),
            "Sens": tp / (tp + fn),
            "Spec": tn / (tn + fp),
            "MCC": matthews_corrcoef(y, preds),
            "Brier": brier_score_loss(y, probs)
        })
    except Exception as e:
        print(f"[ERROR] {name}: {e}")

dfres = pd.DataFrame(results).sort_values(by=["AUC","F1"], ascending=False).reset_index(drop=True)
print(dfres)

[ERROR] LogReg(L2): Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
[ERROR] LDA: Input X contains NaN.
LinearDiscriminantAnalysis does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data,

In [ ]:
# Proper cross-validated
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, matthews_corrcoef, brier_score_loss

WORKDIR = "/content/drive/MyDrive/moltox_project"
meta = pd.read_csv(WORKDIR + "/meta_oof_true_with_gcn.csv")
y = np.load(WORKDIR + "/y.npy")
print("meta shape:", meta.shape, "y shape:", y.shape)

candidates = {
    "LogReg(L2)": LogisticRegression(class_weight="balanced", solver='lbfgs', max_iter=1000),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "MLP_small": MLPClassifier(hidden_layer_sizes=(32,), max_iter=500, random_state=42),
    "RF_meta": RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
    "GB_meta": GradientBoostingClassifier(n_estimators=200, random_state=42),
    "NaiveBayes": GaussianNB()
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []
for name, clf in candidates.items():
    aucs = []; accs = []; f1s = []; sens = []; specs = []; mccs = []; briers = []
    for tr, val in skf.split(meta, y):
        clf.fit(meta.iloc[tr], y[tr])
        if hasattr(clf, "predict_proba"):
            probs = clf.predict_proba(meta.iloc[val])[:,1]
        else:
            probs = clf.predict(meta.iloc[val]).astype(float)
        preds = (probs >= 0.5).astype(int)

        aucs.append(roc_auc_score(y[val], probs))
        accs.append(accuracy_score(y[val], preds))
        f1s.append(f1_score(y[val], preds))
        tn, fp, fn, tp = confusion_matrix(y[val], preds).ravel()
        sens.append(tp/(tp+fn) if (tp+fn)>0 else 0)
        specs.append(tn/(tn+fp) if (tn+fp)>0 else 0)
        mccs.append(matthews_corrcoef(y[val], preds))
        briers.append(brier_score_loss(y[val], probs))

    results.append({
        "name": name,
        "AUC": np.mean(aucs),
        "ACC": np.mean(accs),
        "F1": np.mean(f1s),
        "Sens": np.mean(sens),
        "Spec": np.mean(specs),
        "MCC": np.mean(mccs),
        "Brier": np.mean(briers)
    })

dfres = pd.DataFrame(results).sort_values(by=["AUC","F1"], ascending=False).reset_index(drop=True)
print(dfres.to_string(index=False, float_format="%.4f"))

meta shape: (4062, 4) y shape: (4062,)


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:

import os, numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, matthews_corrcoef, brier_score_loss

WORKDIR = "/content/drive/MyDrive/moltox_project"
META_FILE = os.path.join(WORKDIR, "meta_oof_true_with_gcn.csv")
Y_PATH = os.path.join(WORKDIR, "y.npy")
OUT_CLEAN = os.path.join(WORKDIR, "meta_oof_true_with_gcn.cleaned.csv")

# 0) Load
meta = pd.read_csv(META_FILE)
y = np.load(Y_PATH)
print("Loaded meta shape:", meta.shape, "y shape:", y.shape)
print("Columns:", list(meta.columns))

# 1) NaN diagnostics
nan_counts = meta.isna().sum()
print("\nNaN counts per column:\n", nan_counts.to_string())

rows_with_any_nan = meta.isna().any(axis=1)
n_nan_rows = rows_with_any_nan.sum()
frac_nan = n_nan_rows / len(meta)
print(f"\nRows with any NaN: {n_nan_rows} / {len(meta)}  (fraction = {frac_nan:.4f})")

if n_nan_rows > 0:
    print("\nSample rows with NaN (first 10):")
    display(meta[rows_with_any_nan].head(10))

    bad_idx = np.where(rows_with_any_nan)[0].tolist()
    print("Bad row indices (first 30):", bad_idx[:30])

THRESH = 0.05
action = None
if n_nan_rows == 0:
    action = "none"
elif frac_nan <= THRESH:
    action = "drop"
else:
    if 'gcn_prob' in meta.columns:
        other_cols = [c for c in meta.columns if c.endswith('_prob') and c != 'gcn_prob']
        if len(other_cols) >= 2:
            action = "impute_gcn"
        else:
            action = "drop"
    else:
        action = "drop"

print("\nChosen action ->", action)

meta_clean = meta.copy()
if action == "drop":
    print("Dropping rows with NaN from both meta and y (will reduce dataset size).")
    keep_mask = ~rows_with_any_nan
    meta_clean = meta_clean[keep_mask].reset_index(drop=True)
    y_clean = y[keep_mask]
elif action == "impute_gcn":
    print("Imputing gcn_prob NaNs with mean of other base probabilities (per-row mean).")
    base_cols = [c for c in meta_clean.columns if c.endswith('_prob') and c != 'gcn_prob']
    row_mean = meta_clean[base_cols].mean(axis=1)
    meta_clean['gcn_prob'] = meta_clean['gcn_prob'].fillna(row_mean)
    y_clean = y.copy()
elif action == "none":
    print("No NaNs found; proceeding without modification.")
    meta_clean = meta.copy()
    y_clean = y.copy()
else:
    raise RuntimeError("Unknown action")

print("After remediation meta_clean shape:", meta_clean.shape, "y_clean shape:", y_clean.shape)
meta_clean.to_csv(OUT_CLEAN, index=False)
print("Saved cleaned meta matrix to:", OUT_CLEAN)

print("\nPost-clean NaN counts:\n", meta_clean.isna().sum().to_string())

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
candidates = {
    "LogReg(L2)": LogisticRegression(class_weight="balanced", solver='lbfgs', max_iter=1000),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "MLP_small": MLPClassifier(hidden_layer_sizes=(32,), max_iter=500, random_state=42),
    "RF_meta": RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
    "GB_meta": GradientBoostingClassifier(n_estimators=200, random_state=42),
    "NaiveBayes": GaussianNB()
}

results = []
X = meta_clean
Y = y_clean
for name, clf in candidates.items():
    aucs, accs, f1s, sens, specs, mccs, briers = [], [], [], [], [], [], []
    for tr, val in skf.split(X, Y):
        clf.fit(X.iloc[tr], Y[tr])
        if hasattr(clf, "predict_proba"):
            probs = clf.predict_proba(X.iloc[val])[:,1]
        else:
            probs = clf.predict(X.iloc[val]).astype(float)
        preds = (probs >= 0.5).astype(int)

        try:
            aucs.append(roc_auc_score(Y[val], probs))
        except Exception:
            aucs.append(float('nan'))
        accs.append(accuracy_score(Y[val], preds))
        f1s.append(f1_score(Y[val], preds))
        tn, fp, fn, tp = confusion_matrix(Y[val], preds).ravel()
        sens.append(tp/(tp+fn) if (tp+fn)>0 else 0)
        specs.append(tn/(tn+fp) if (tn+fp)>0 else 0)
        mccs.append(matthews_corrcoef(Y[val], preds))
        briers.append(brier_score_loss(Y[val], probs))

    results.append({
        "name": name,
        "AUC": np.nanmean(aucs),
        "ACC": np.mean(accs),
        "F1": np.mean(f1s),
        "Sens": np.mean(sens),
        "Spec": np.mean(specs),
        "MCC": np.mean(mccs),
        "Brier": np.mean(briers)
    })
    print("Completed meta:", name)

import pandas as pd
dfres = pd.DataFrame(results).sort_values(by=["AUC","F1"], ascending=False).reset_index(drop=True)
print("\nMeta-learner comparison AFTER NaN remediation:")
print(dfres.to_string(index=False, float_format='%.4f'))

if dfres[['AUC','ACC','F1']].max().max() >= 0.9999:
    print("\n***** ALERT: some metrics are ~1.0 after cleaning. Run the earlier diagnostics to check for leakage. *****")

Loaded meta shape: (4062, 4) y shape: (4062,)
Columns: ['rf_prob', 'lgbm_prob', 'mlp_prob', 'gcn_prob']

NaN counts per column:
 rf_prob         0
lgbm_prob       0
mlp_prob        0
gcn_prob     4062

Rows with any NaN: 4062 / 4062  (fraction = 1.0000)

Sample rows with NaN (first 10):


,rf_prob,lgbm_prob,mlp_prob,gcn_prob
0,0.505,0.054440,0.000788,NaN
1,0.395,0.074604,0.976633,NaN
2,0.280,0.059310,0.067400,NaN
3,0.625,0.391126,0.999629,NaN
4,0.460,0.304130,0.858056,NaN
5,0.505,0.292985,0.127587,NaN
6,0.425,0.012757,0.000014,NaN
7,0.875,0.975748,0.999894,NaN
8,0.780,0.965487,0.998715,NaN
9,0.520,0.206818,0.983485,NaN


Bad row indices (first 30): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]

Chosen action -> impute_gcn
Imputing gcn_prob NaNs with mean of other base probabilities (per-row mean).
After remediation meta_clean shape: (4062, 4) y_clean shape: (4062,)
Saved cleaned meta matrix to: /content/drive/MyDrive/moltox_project/meta_oof_true_with_gcn.cleaned.csv

Post-clean NaN counts:
 rf_prob      0
lgbm_prob    0
mlp_prob     0
gcn_prob     0
Completed meta: LogReg(L2)
Completed meta: LDA
Completed meta: QDA


/usr/local/lib/python3.12/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/p

Completed meta: MLP_small
Completed meta: RF_meta
Completed meta: GB_meta
Completed meta: NaiveBayes

Meta-learner comparison AFTER NaN remediation:
      name    AUC    ACC     F1   Sens   Spec    MCC  Brier
       LDA 0.9468 0.8772 0.8988 0.8998 0.8422 0.7429 0.1007
   GB_meta 0.9462 0.8735 0.8966 0.9043 0.8260 0.7343 0.0909
 MLP_small 0.9450 0.8779 0.8999 0.9051 0.8360 0.7440 0.0901
LogReg(L2) 0.9449 0.8779 0.8982 0.8876 0.8629 0.7465 0.0925
NaiveBayes 0.9427 0.8712 0.8922 0.8783 0.8604 0.7334 0.1183
       QDA 0.9421 0.8754 0.8952 0.8767 0.8735 0.7434 0.1063
   RF_meta 0.9406 0.8717 0.8954 0.9043 0.8216 0.7304 0.0940


In [ ]:
import os, traceback, joblib
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, matthews_corrcoef, brier_score_loss

WORKDIR = "/content/drive/MyDrive/moltox_project"
META_IN = os.path.join(WORKDIR, "meta_oof_true_with_gcn.csv")
Y_PATH  = os.path.join(WORKDIR, "y.npy")
CLEAN_META_OUT = os.path.join(WORKDIR, "meta_oof_true_with_gcn.cleaned.csv")
CLEAN_Y_OUT = os.path.join(WORKDIR, "y_clean.npy")
BEST_META_OUT = os.path.join(WORKDIR, "best_meta_after_clean.joblib")
LOGFILE = os.path.join(WORKDIR, "final_wrapup.log")

def log(msg):
    print(msg)

try:
    log("1) Loading meta and labels...")
    meta = pd.read_csv(META_IN)
    y = np.load(Y_PATH)
    log(f"   meta shape: {meta.shape}   y shape: {y.shape}")
    log("   columns: " + ", ".join(meta.columns.tolist()))

    log("\n2) NaN diagnostics")
    nan_counts = meta.isna().sum()
    log("   NaN counts per column:\n" + nan_counts.to_string())
    rows_with_any_nan = meta.isna().any(axis=1)
    n_nan_rows = int(rows_with_any_nan.sum())
    frac_nan = n_nan_rows / len(meta)
    log(f"   Rows with any NaN: {n_nan_rows} / {len(meta)} (fraction {frac_nan:.4f})")
    if n_nan_rows > 0:
        log("   Example rows with NaN (first 6):")
        display(meta[rows_with_any_nan].head(6))

    DROP_THRESHOLD = 0.05
    action = None
    if n_nan_rows == 0:
        action = "none"
    else:
        if frac_nan <= DROP_THRESHOLD:
            action = "drop"
        else:
            action = "impute_gcn"
    log(f"\n3) Chosen remediation action: {action}")

    meta_clean = meta.copy()
    y_clean = y.copy()
    if action == "none":
        log("   No NaNs found; nothing to remediate.")
    elif action == "drop":
        log("   Dropping rows with NaNs (safer when only a few rows missing).")
        keep_mask = ~rows_with_any_nan
        meta_clean = meta_clean[keep_mask].reset_index(drop=True)
        y_clean = y[keep_mask]
        np.save(CLEAN_Y_OUT, y_clean)    # save cleaned y
        log(f"   Dropped {n_nan_rows} rows; new meta shape: {meta_clean.shape}; saved cleaned y to {CLEAN_Y_OUT}")
    elif action == "impute_gcn":
        log("   Imputing gcn_prob NaNs using per-row mean of other base probabilities.")
        if "gcn_prob" not in meta_clean.columns:
            raise RuntimeError("gcn_prob column missing; cannot impute.")
        base_cols = [c for c in meta_clean.columns if c.endswith("_prob") and c != "gcn_prob"]
        if len(base_cols) < 1:
            raise RuntimeError("Not enough base prob columns for imputation.")
        row_mean = meta_clean[base_cols].mean(axis=1)
        meta_clean["gcn_prob"] = meta_clean["gcn_prob"].fillna(row_mean)
        log(f"   Imputed gcn_prob using columns: {base_cols}")
    else:
        raise RuntimeError("Unknown remediation action")

    meta_clean.to_csv(CLEAN_META_OUT, index=False)
    np.save(CLEAN_Y_OUT, y_clean)
    log(f"4) Saved cleaned meta to: {CLEAN_META_OUT}")
    log(f"   Saved cleaned y to: {CLEAN_Y_OUT}")

    nan_counts_post = meta_clean.isna().sum()
    log("\n5) Post-clean NaN counts per column:\n" + nan_counts_post.to_string())
    if meta_clean.shape[0] != len(y_clean):
        raise RuntimeError(f"Length mismatch after cleaning: meta rows {meta_clean.shape[0]} vs y {len(y_clean)}")

    log("\n6) Evaluating meta-learners via StratifiedKFold CV (5 splits)...")
    X = meta_clean
    Y = y_clean
    # Candidate meta models
    candidates = {
        "LogReg(L2)": LogisticRegression(class_weight="balanced", solver='lbfgs', max_iter=1000),
        "LDA": LinearDiscriminantAnalysis(),
        "QDA": QuadraticDiscriminantAnalysis(),
        "MLP_small": MLPClassifier(hidden_layer_sizes=(32,), max_iter=500, random_state=42),
        "RF_meta": RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
        "GB_meta": GradientBoostingClassifier(n_estimators=200, random_state=42),
        "NaiveBayes": GaussianNB()
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    results = []
    for name, clf in candidates.items():
        aucs, accs, f1s, sens, specs, mccs, briers = [], [], [], [], [], [], []
        for tr, val in skf.split(X, Y):
            clf.fit(X.iloc[tr], Y[tr])
            if hasattr(clf, "predict_proba"):
                probs = clf.predict_proba(X.iloc[val])[:,1]
            else:
                probs = clf.predict(X.iloc[val]).astype(float)
            preds = (probs >= 0.5).astype(int)

            # Safe metrics
            try:
                aucs.append(roc_auc_score(Y[val], probs))
            except Exception:
                aucs.append(float('nan'))
            accs.append(accuracy_score(Y[val], preds))
            f1s.append(f1_score(Y[val], preds))
            tn, fp, fn, tp = confusion_matrix(Y[val], preds).ravel()
            sens.append(tp/(tp+fn) if (tp+fn)>0 else 0.0)
            specs.append(tn/(tn+fp) if (tn+fp)>0 else 0.0)
            mccs.append(matthews_corrcoef(Y[val], preds))
            briers.append(brier_score_loss(Y[val], probs))

        results.append({
            "name": name,
            "AUC": np.nanmean(aucs),
            "ACC": np.mean(accs),
            "F1": np.mean(f1s),
            "Sens": np.mean(sens),
            "Spec": np.mean(specs),
            "MCC": np.mean(mccs),
            "Brier": np.mean(briers)
        })
        log(f"   Completed meta: {name}")

    dfres = pd.DataFrame(results).sort_values(by=["AUC","F1"], ascending=False).reset_index(drop=True)
    log("\nMeta-learner comparison (CV results):")
    print(dfres.to_string(index=False, float_format="%.4f"))

    best_name = dfres.loc[0, "name"]
    best_clf = candidates[best_name]
    best_clf.fit(X, Y)
    joblib.dump(best_clf, BEST_META_OUT)
    log(f"\n7) Saved best meta model: {best_name} -> {BEST_META_OUT}")

    max_metric = dfres[['AUC','ACC','F1']].max().max()
    if max_metric >= 0.9999:
        log("\n*** WARNING: some metrics are ~1.0 after cleaning — investigate leakage. ***")
    else:
        log("\nAll good: no perfect-metric alert.")

    log("\n=== SUMMARY (copy into slides) ===")
    print(f"- Issue detected: {n_nan_rows} rows had NaN values in meta (fraction {frac_nan:.4f}).")
    if action == "drop":
        print("- Remediation: dropped rows with NaN (safe when few missing).")
    elif action == "impute_gcn":
        print("- Remediation: imputed missing gcn_prob with the per-row mean of other base model probabilities (conservative).")
    else:
        print("- Remediation: none required.")

    print(f"- Final meta evaluation (top rows):\n{dfres.head(5).to_string(index=False)}")
    print("\nSuggested next steps (short bullets):")
    print("  1) Re-generate true GCN OOF predictions (replace imputation) by re-running the GCN OOF cell.")
    print("  2) Tune GCN & meta hyperparameters (grid/optuna) and try other graph models (MPNN, GAT, AttentiveFP).")
    print("  3) Keep a held-out test set (10-20%) untouched until final evaluation for the author/benchmarks.")
    print("  4) For slides: show 'problem -> fix -> result' flow, and include the CV table above.")

    log("\nDone. Cleaned files saved:")
    log(" - cleaned meta: " + CLEAN_META_OUT)
    log(" - cleaned y:    " + CLEAN_Y_OUT)
    log(" - best meta:    " + BEST_META_OUT)

except Exception as e:
    with open(LOGFILE, "a") as lf:
        lf.write("Exception in final wrapup:\n")
        lf.write(traceback.format_exc() + "\n")
    print("ERROR during final wrapup. Traceback saved to", LOGFILE)
    raise

1) Loading meta and labels...
   meta shape: (4062, 4)   y shape: (4062,)
   columns: rf_prob, lgbm_prob, mlp_prob, gcn_prob

2) NaN diagnostics
   NaN counts per column:
rf_prob         0
lgbm_prob       0
mlp_prob        0
gcn_prob     4062
   Rows with any NaN: 4062 / 4062 (fraction 1.0000)
   Example rows with NaN (first 6):


,rf_prob,lgbm_prob,mlp_prob,gcn_prob
0,0.505,0.054440,0.000788,NaN
1,0.395,0.074604,0.976633,NaN
2,0.280,0.059310,0.067400,NaN
3,0.625,0.391126,0.999629,NaN
4,0.460,0.304130,0.858056,NaN
5,0.505,0.292985,0.127587,NaN



3) Chosen remediation action: impute_gcn
   Imputing gcn_prob NaNs using per-row mean of other base probabilities.
   Imputed gcn_prob using columns: ['rf_prob', 'lgbm_prob', 'mlp_prob']
4) Saved cleaned meta to: /content/drive/MyDrive/moltox_project/meta_oof_true_with_gcn.cleaned.csv
   Saved cleaned y to: /content/drive/MyDrive/moltox_project/y_clean.npy

5) Post-clean NaN counts per column:
rf_prob      0
lgbm_prob    0
mlp_prob     0
gcn_prob     0

6) Evaluating meta-learners via StratifiedKFold CV (5 splits)...
   Completed meta: LogReg(L2)
   Completed meta: LDA


/usr/local/lib/python3.12/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/p

   Completed meta: QDA
   Completed meta: MLP_small
   Completed meta: RF_meta
   Completed meta: GB_meta
   Completed meta: NaiveBayes

Meta-learner comparison (CV results):
      name    AUC    ACC     F1   Sens   Spec    MCC  Brier
       LDA 0.9468 0.8772 0.8988 0.8998 0.8422 0.7429 0.1007
   GB_meta 0.9462 0.8735 0.8966 0.9043 0.8260 0.7343 0.0909
 MLP_small 0.9450 0.8779 0.8999 0.9051 0.8360 0.7440 0.0901
LogReg(L2) 0.9449 0.8779 0.8982 0.8876 0.8629 0.7465 0.0925
NaiveBayes 0.9427 0.8712 0.8922 0.8783 0.8604 0.7334 0.1183
       QDA 0.9421 0.8754 0.8952 0.8767 0.8735 0.7434 0.1063
   RF_meta 0.9406 0.8717 0.8954 0.9043 0.8216 0.7304 0.0940

7) Saved best meta model: LDA -> /content/drive/MyDrive/moltox_project/best_meta_after_clean.joblib

All good: no perfect-metric alert.

=== SUMMARY (copy into slides) ===
- Issue detected: 4062 rows had NaN values in meta (fraction 1.0000).
- Remediation: imputed missing gcn_prob with the per-row mean of other base model probabilities (conse

In [ ]:
import joblib
os.makedirs(os.path.join(WORKDIR,"models"), exist_ok=True)
joblib.dump(best_rf, os.path.join(WORKDIR,"models","best_rf.joblib"))
joblib.dump(best_lgb, os.path.join(WORKDIR,"models","best_lgb.joblib"))
joblib.dump(best_mlp, os.path.join(WORKDIR,"models","best_mlp.joblib"))
joblib.dump(meta, os.path.join(WORKDIR,"models","meta_logreg.joblib"))
joblib.dump(scaler, os.path.join(WORKDIR,"models","scaler.joblib"))
pd.DataFrame(meta_train, columns=['rf_prob','lgbm_prob','mlp_prob']).to_csv(os.path.join(WORKDIR,"meta_train_oof.csv"), index=False)
print("Saved models to", os.path.join(WORKDIR,"models"))


In [ ]:
import os

workdir = "/content/drive/MyDrive/moltox_project"
for f in os.listdir(workdir):
    print(f)
